In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

In [22]:
n_input = 784
n_classes = 10
n_hidden_layer = 256
learning_rate = 0.001
batch_size = 128
epochs = 5

In [11]:
features = tf.placeholder(dtype=tf.float32, shape=[None, n_input])
labels = tf.placeholder(dtype=tf.float32, shape=[None, n_classes])

In [12]:
weights = {
    "hidden_layer": tf.Variable(tf.random_normal(shape=[n_input, n_hidden_layer])),
    "out_layer": tf.Variable(tf.random_normal(shape=[n_hidden_layer, n_classes]))
}
bias = {
    "hidden_layer": tf.Variable(tf.zeros(shape=[n_hidden_layer])),
    "out_layer": tf.Variable(tf.zeros(shape=[n_classes]))
}

In [13]:
hidden_layer_input = tf.add(tf.matmul(features, weights['hidden_layer']), bias['hidden_layer'])
hidden_layer_output = tf.nn.relu(hidden_layer_input)

logits = tf.add(tf.matmul(hidden_layer_output, weights['out_layer']), bias['out_layer'])

In [14]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels = labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [15]:
# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [16]:
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    # TODO: Implement batching
    output_batches = []
    for start_i in range(0, len(features), batch_size):
        end_i = start_i+batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        output_batches.append(batch)
    return output_batches

In [17]:
def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(cost, feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(accuracy, feed_dict={features: valid_features, labels: valid_labels})
    
    print("Epoch: {:<4} - Cost: {:<8.3} Valid accuracy: {:<8.3}".format(epoch_i, current_cost, valid_accuracy))

In [24]:
save_path = './Data/mnist_v0.2.ckpt'
saver = tf.train.Saver()

In [25]:
assert batch_size is not None, 'You must set the batch size.'

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch_i in range(epochs):
        for batch_features, batch_labels in batches(batch_size, train_features, train_labels):
            sess.run(optimizer, feed_dict={features:batch_features, labels:batch_labels})
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)
    saver.save(sess, save_path)
    test_accuracy = sess.run(accuracy, feed_dict={features:test_features, labels:test_labels})
print('Test Accuracy: {}'.format(test_accuracy))

Epoch: 0    - Cost: 38.2     Valid accuracy: 0.415   
Epoch: 1    - Cost: 24.5     Valid accuracy: 0.556   
Epoch: 2    - Cost: 19.8     Valid accuracy: 0.627   
Epoch: 3    - Cost: 17.3     Valid accuracy: 0.671   
Epoch: 4    - Cost: 15.7     Valid accuracy: 0.705   
Test Accuracy: 0.7092999815940857


In [26]:
with tf.Session() as sess:
    saver.restore(sess, save_path)
    test_accuracy = sess.run(accuracy, feed_dict={features:test_features, labels:test_labels})
print('Test Accuracy: {}'.format(test_accuracy))

INFO:tensorflow:Restoring parameters from ./Data/mnist_v0.2.ckpt
Test Accuracy: 0.7092999815940857
